# Anem a resoldre l'exemplre de viatjar per les ciutats de Romania

Has de buscar el camí més curt per anar de Arad a Bucharest. Per això has de crear un graf amb les ciutats i les distàncies entre elles. Les distàncies les pots trobar al diccionari `distancies` que ja està creat.

Has d'utilitzar la metodologia que ja hem vist a classe, creant l'estat `EstatRomania` i el problema ProblemaRomania. Després has de resoldre el problema amb ucs i A*.

Et proporcionem la classe Problema, que ja hem utilitzat, el diccionari de distàncies i un altre diccionari amb les distàncies en línia recta entre les ciutats i Bucharest.


In [197]:
distancies = {
    'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Pitesti': {'Rimnicu Vilcea': 97, 'Craiova': 138, 'Bucharest': 101},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Giurgiu': 90},
    'Giurgiu': {'Bucharest': 90}
}

distancies_recta = {
    'Arad': 366,
    'Zerind': 374,
    'Oradea': 380,
    'Sibiu': 253,
    'Timisoara': 329,
    'Lugoj': 244,
    'Mehadia': 241,
    'Drobeta': 242,
    'Craiova': 160,
    'Rimnicu Vilcea': 193,
    'Fagaras': 176,
    'Pitesti': 100,
    'Bucharest': 0,
    'Giurgiu': 77
}


In [198]:
class Problema(object):
    """Aquesta és la classe abstracta per a un problema formal. Una nova àrea crea una subclasse d'aquesta, sobrescrivint `accions` i `accio`, i potser altres mètodes.
    L'heurística per defecte és 0 i el cost d'acció per defecte és 1 per a tots els estats.
    Quan crees una instància d'una subclasse, especifica els estats `inicial` i `final`
    (o proporciona un mètode `es_resultat`) i potser altres arguments de paraula clau per a la subclasse."""

    def __init__(self, inicial=None, final=None, **kwds):
        self.__dict__.update(inicial=inicial, final=final, **kwds)

    def accions(self, state):         raise NotImplementedError

    def accio(self, state, action):   raise NotImplementedError

    def es_resultat(self, state):     return state == self.final

    def cost_accio(self, s, a, s1):   return 1

    def h(self, estat):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.inicial, self.final)

In [199]:
from dataclasses import dataclass


@dataclass
class EstatRomania(object):
    """Aquesta classe representa un estat del problema de Romania. L'estat és una ciutat."""

    ciutat: str
    pare: 'EstatRomania' = None

    def __repr__(self):
        return 'EstatRomania({!r})'.format(self.ciutat)

    def __str__(self):
        return '{}'.format(self.ciutat)

    def __eq__(self, other):
        return self.ciutat == other.ciutat

    def __hash__(self):
        return hash(self.ciutat)

    def __lt__(self, other):
        return self.ciutat < other.ciutat

In [200]:
class ProblemaRomania(Problema):
    """Aquesta classe representa el problema de Romania. L'estat és una ciutat."""

    def __init__(self, inicial, final):
        super().__init__(inicial, final)

    def accions(self, estat):
        """Retorna una llista d'accions possibles a l'estat actual."""
        return list(distancies[estat.ciutat].keys())

    def accio(self, estat, accio):
        """Retorna l'estat que resulta d'aplicar l'accio a l'estat."""
        return EstatRomania(accio, estat)

    def cost_accio(self, estat, accio, estat1):
        """Retorna el cost d'aplicar l'accio a l'estat."""
        return distancies[estat.ciutat][accio]

    def h(self, estat):
        """Retorna una estimació heurística del cost per anar de l'estat a l'estat final."""
        return distancies_recta[estat.ciutat]

In [201]:
import heapq

def ucs(problema):
    """Algorisme de cerca uniforme. Retorna una llista amb el camí des de l'estat inicial fins a l'estat final."""
    frontera = [(0, problema.inicial)]
    visitats = set()
    while frontera:
        cost, estat = heapq.heappop(frontera)
        if problema.es_resultat(estat):
            print('Visitats:', len(visitats))
            return estat
        if estat not in visitats:
            visitats.add(estat)
            for accio in problema.accions(estat):
                estat1 = problema.accio(estat, accio)
                cost1 = cost + problema.cost_accio(estat, accio, estat1)
                heapq.heappush(frontera, (cost1, estat1))
        

In [202]:
def extreu_ruta(estat):
    """Retorna una llista amb els estats de la ruta des de l'estat inicial fins a l'estat final."""
    ruta = []
    while estat.pare:
        ruta.append(estat)
        estat = estat.pare
    ruta.append(estat)
    ruta.reverse()
    return ruta

def mostra_ruta(estat):
    """Mostra la ruta des de l'estat inicial fins a l'estat final."""
    ruta = extreu_ruta(estat)
    cost = 0
    for estat in ruta:
        if estat.pare:
            print(f"{estat.pare} -> {estat}  ({distancies[estat.pare.ciutat][estat.ciutat]})")
            cost += distancies[estat.pare.ciutat][estat.ciutat]
        else:
            print(estat)

    print(f"Cost: {cost}")

In [203]:
problema = ProblemaRomania(EstatRomania('Arad'), EstatRomania('Bucharest'))
estat = ucs(problema)

mostra_ruta(estat)

Visitats: 12
Arad
Arad -> Sibiu  (140)
Sibiu -> Rimnicu Vilcea  (80)
Rimnicu Vilcea -> Pitesti  (97)
Pitesti -> Bucharest  (101)
Cost: 418


In [204]:
def a_estrella(problema):
    """Algorisme de cerca A*. Retorna una llista amb el camí des de l'estat inicial fins a l'estat final."""
    frontera = [(problema.h(problema.inicial), problema.inicial)]
    visitats = set()
    while frontera:
        cost, estat = heapq.heappop(frontera)
        if problema.es_resultat(estat):
            print('Visitats:', len(visitats))
            return estat
        if estat not in visitats:
            visitats.add(estat)
            for accio in problema.accions(estat):
                estat1 = problema.accio(estat, accio)
                cost1 = cost + problema.cost_accio(estat, accio, estat1)
                heapq.heappush(frontera, (cost1 + problema.h(estat1), estat1))


In [205]:
problema = ProblemaRomania(EstatRomania('Arad'), EstatRomania('Bucharest'))
estat = a_estrella(problema)

mostra_ruta(estat)


Visitats: 8
Arad
Arad -> Sibiu  (140)
Sibiu -> Fagaras  (99)
Fagaras -> Bucharest  (211)
Cost: 450


# Questió

Com haurás observat el resultat de la cerca uniforme i de la cerca A* no és el mateix. Quina creus que pot ser la causa? Que hauriem de fer per arreglar-ho?